In [8]:
import numpy as np 
import pandas as pd 

# To consume Twitter's API
import tweepy
from tweepy import OAuthHandler 

In [9]:
# keys and tokens from the Twitter Dev Console
consumer_key = 'Sec3MvclRIx2RVlgu9l0SJX6D'
consumer_secret = 'ayoPNWtBm7fWpMBoK6EwRmegu3SW8Rw9mzJkottkv97quPe941'
access_token = '736550752760406018-so5CPJrEbJKb3c3Pq8va3VFr0yk4S0E'
access_token_secret = 'Cgr8tz0h6FTU7kxAjDzpHnjffNTHxWsBytXnu4Ihd1TFb'

In [10]:
class TwitterClient(object): 
    def __init__(self): 
        #Initialization method. 
        try: 
            # create OAuthHandler object 
            auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            # add hyper parameter 'proxy' if executing from behind proxy "proxy='http://172.22.218.218:8085'"
            self.api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
            
        except tweepy.TweepError as e:
            print(f"Error: Tweeter Authentication Failed - \n{str(e)}")

    def get_tweets(self, query, maxTweets = 1000):
        #Function to fetch tweets. 
        # empty list to store parsed tweets 
        tweets = [] 
        sinceId = None
        max_id = -1
        tweetCount = 0
        tweetsPerQry = 100

        while tweetCount < maxTweets:
            try:
                if (max_id <= 0):
                    if (not sinceId):
                        new_tweets = self.api.search(q=query, count=tweetsPerQry)
                    else:
                        new_tweets = self.api.search(q=query, count=tweetsPerQry,
                                                since_id=sinceId)
                else:
                    if (not sinceId):
                        new_tweets = self.api.search(q=query, count=tweetsPerQry,
                                                max_id=str(max_id - 1))
                    else:
                        new_tweets = self.api.search(q=query, count=tweetsPerQry,
                                                max_id=str(max_id - 1),
                                                since_id=sinceId)
                if not new_tweets:
                    print("No more tweets found")
                    break

                for tweet in new_tweets:
                    parsed_tweet = {} 
                    parsed_tweet['tweets'] = tweet.text 

                    # appending parsed tweet to tweets list 
                    if tweet.retweet_count > 0: 
                        # if tweet has retweets, ensure that it is appended only once 
                        if parsed_tweet not in tweets: 
                            tweets.append(parsed_tweet) 
                    else: 
                        tweets.append(parsed_tweet) 
                        
                tweetCount += len(new_tweets)
                print("Downloaded {0} tweets".format(tweetCount))
                max_id = new_tweets[-1].id

            except tweepy.TweepError as e:
                # Just exit if any error
                print("Tweepy error : " + str(e))
                break
        
        return pd.DataFrame(tweets)

## (A). Mining Latest trending topic for India from twitter

In [14]:
twitter_client = TwitterClient()

# calling function to get tweets
tweets_df = twitter_client.get_tweets('Trending India', maxTweets=100)
print(f'tweets_df Shape - {tweets_df.shape}')
tweets_df.head(10)

Downloaded 100 tweets
tweets_df Shape - (30, 1)


,tweets
0,RT @TPHOffl: #PrabhasNagAshwin Is Trending In ...
1,RT @Sids_Warrior14: Our Tag #TimesManOfTheYear...
2,RT @PremSha23719235: Just have look at Trendli...
3,RT @NBK_MB_cult: #SarkaruVaariPaata trending a...
4,RT @Yeruvaka99: #SarkaruVaariPaata Hashtag Tre...
5,RT @SidShukla_1: In just few mins it's trendin...
6,Our Tag #TimesManOfTheYearSidharth is trending...
7,RT @Yeruvaka99: #PrabhasNagAshwin Hashtag is T...
8,RT @Yeruvaka99: #SarkaruVaariPaata Hashtag Tre...
9,RT @Yeruvaka99: #NTR30 Hashtag Trending on #Tw...


## (B). Mining 100+ tweets for #JoeBiden from Twitter and Sentiment Analysis

In [15]:
twitter_client = TwitterClient()

# calling function to get tweets
tweets_df = twitter_client.get_tweets('#JoeBiden', maxTweets=100)
print(f'tweets_df Shape - {tweets_df.shape}')
tweets_df.head(10)

Downloaded 96 tweets
Downloaded 192 tweets
tweets_df Shape - (157, 1)


,tweets
0,"In #JoeBiden, the Health Care Industry Has Fou..."
1,#JoeBiden act now and save children of Tigray ...
2,RT @DainikBhaskar: तरजीह का अंदाजा: बतौर राष्ट...
3,RT @WilkowMajority: The left wing media is cel...
4,RT @Shawn615923502: @DvObserver @CBSNews @Reut...
5,"How to achieve ""De-Trumpification""\n\nhttps://..."
6,RT @auntbarb73: #JoeBiden #NancyPelosi #Kamala...
7,RT @elliielle: Dear President @POTUS and @VP...
8,RT @UrduNewsCom: وائٹ ہاؤس میں ابتدائی 10 دنوں...
9,RT @elliielle: @MHackman Dear President @POTUS...


In [12]:
def fetch_sentiment_using_SIA(text):
    sid = SentimentIntensityAnalyzer()
    polarity_scores = sid.polarity_scores(text)
    return 'neg' if polarity_scores['neg'] > polarity_scores['pos'] else 'pos'

In [13]:
sentiments_using_SIA = tweets_df.tweets.apply(lambda tweet: fetch_sentiment_using_SIA(tweet))
pd.DataFrame(sentiments_using_SIA.value_counts())

,tweets
pos,121
neg,24
